In [123]:
using LinearAlgebra, SparseArrays

In [124]:
using LowLevelFEM
gmsh.initialize()

gmsh.open("tarto.geo")

Info    : Reading 'tarto.geo'...
Info    : Meshing 1D...- Splitting solids                                                                                             
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (TrimmedCurve)
Info    : [ 10%] Meshing curve 6 (TrimmedCurve)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (TrimmedCurve)
Info    : [ 10%] Meshing curve 10 (TrimmedCurve)
Info    : [ 20%] Meshing curve 11 (Line)
Info    : [ 20%] Meshing curve 12 (Circle)
Info    : [ 20%] Meshing curve 13 (Circle)
Info    : [ 20%] Meshing curve 14 (Circle)
Info    : [ 20%] Meshing curve 15 (Circle)
Info    : [ 20%] Meshing curve 16 (Line)
Info    : [ 20%] Meshing curve 17 (Circle)
Info    : [ 20%] Meshing curve 18 (Line)
Info    : [ 20%] Meshing curve 19 (Circle)
Info    : [ 20%] M

In [125]:
mat = Material("part")
prob = Problem([mat])

Problem("tarto", :Solid, 3, 3, Material[Material("part", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 27263, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [126]:
supp1 = displacementConstraint("base", uz=0)
supp2 = displacementConstraint("hole1", ux=0, uy=0)
supp3 = displacementConstraint("perim", uz=-0.00001)

load1 = load("hole2", fy=-5);

In [127]:
F = loadVector(prob, [load1])
showDoFResults(F, name="load");

In [128]:
resultant(F, "hole2")

3-element Vector{Float64}:
    0.0
 -942.413625487596
    0.0

In [129]:
λ = mat.λ
μ = mat.μ;

In [130]:
@time K = poissonMatrix(prob, coefficient=2μ) + gradDivMatrix(prob, coefficient=λ) - curlCurlMatrix(prob, coefficient=μ)

  9.349199 seconds (45.99 M allocations: 5.398 GiB, 14.38% gc time)


sparse([1, 2, 3, 169, 170, 171, 223, 224, 225, 280  …  76602, 76603, 76604, 76605, 81709, 81710, 81711, 81787, 81788, 81789], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  81789, 81789, 81789, 81789, 81789, 81789, 81789, 81789, 81789, 81789], [270543.11346134474, -11159.804377662445, 102644.77760057495, -4597.344962031356, 264.3247288412763, -4646.554931874546, 5175.806547185506, -5921.919862186506, 11334.867428781494, 20476.172811385757  …  -298334.2519184167, -186029.82034847172, -224222.7973486119, -280188.98738505895, -59856.25414105746, 77623.63039804924, 112989.45363675174, -107045.1003009175, 270525.9942690422, 1.5092618746204376e6], 81789, 81789)

In [131]:
@time K1 = symmetricGradientMatrix(prob, coefficient=μ) + gradDivMatrix(prob, coefficient=λ)

  5.257838 seconds (30.66 M allocations: 3.578 GiB, 12.43% gc time)


sparse([1, 2, 3, 169, 170, 171, 223, 224, 225, 280  …  76602, 76603, 76604, 76605, 81709, 81710, 81711, 81787, 81788, 81789], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  81789, 81789, 81789, 81789, 81789, 81789, 81789, 81789, 81789, 81789], [270543.11346134474, -11159.804377662445, 102644.77760057495, -4597.344962031355, 264.3247288412763, -4646.554931874546, 5175.806547185506, -5921.919862186506, 11334.867428781494, 20476.17281138576  …  -298334.2519184167, -186029.82034847172, -224222.7973486119, -280188.98738505895, -59856.25414105746, 77623.63039804924, 112989.45363675173, -107045.1003009175, 270525.9942690422, 1.5092618746204379e6], 81789, 81789)

In [132]:
@time K0 = stiffnessMatrix(prob)

  3.530018 seconds (17.46 M allocations: 8.493 GiB, 26.64% gc time)


sparse([1, 2, 3, 169, 170, 171, 223, 224, 225, 280  …  76602, 76603, 76604, 76605, 81709, 81710, 81711, 81787, 81788, 81789], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  81789, 81789, 81789, 81789, 81789, 81789, 81789, 81789, 81789, 81789], [270543.11346134485, -11159.80437766245, 102644.777600575, -4597.344962031357, 264.3247288412764, -4646.554931874547, 5175.806547185517, -5921.919862186509, 11334.867428781497, 20476.17281138576  …  -298334.2519184168, -186029.82034847175, -224222.79734861193, -280188.98738505895, -59856.25414105749, 77623.63039804928, 112989.45363675182, -107045.10030091746, 270525.9942690423, 1.5092618746204376e6], 81789, 81789)

In [133]:
norm(K1[:, :] - K0[:, :])

1.0100270493710848e-7

In [134]:
K_theory = poissonMatrix(prob, coefficient=2μ) +
           gradDivMatrix(prob, coefficient=λ) -
           curlCurlMatrix(prob, coefficient=μ)

norm(K_theory[:, :] - K0[:, :])

1.127814762713756e-7

In [135]:
f = loadVector(prob, [load1]);

In [136]:
all = constrainedDoFs(prob, [BoundaryCondition("part", ux=0, uy=0, uz=0)])
bc = constrainedDoFs(prob, [supp1, supp2, supp3])
free = setdiff(all, bc);

In [137]:
u = applyBoundaryConditions(prob, [supp1, supp2, supp3]);

In [138]:
DoFs(u)[free] = K1[free, free] \ DoFs(f)[free];

In [139]:
showDoFResults(u, name="displacement")

1

In [140]:
@time A = (u ∘ ∇ + ∇ ∘ u) / 2

  3.884216 seconds (17.27 M allocations: 1.425 GiB, 5.93% gc time)


elementwise TensorField
[[3.3831020691491816e-7; 5.410474984735239e-7; … ; -1.1957431261516455e-5; 3.5564953637996235e-5;;], [2.987018387618966e-6; 1.8592654330836968e-5; … ; -1.9107435703011883e-5; 4.266973601557367e-5;;], [1.3064995123031418e-5; -2.8819977457605114e-5; … ; -2.5032479062644074e-5; -0.0001984178011544182;;], [1.7127378517020313e-5; 4.5873824626422556e-6; … ; -1.133300565141751e-6; 6.447111163459922e-7;;], [-6.82533436350216e-6; 2.3328215026195404e-7; … ; -1.952869279938182e-5; 0.00014562220978882128;;], [-6.186684693512123e-6; 5.102796813583812e-7; … ; -1.918832683870048e-5; 0.00015913039930430658;;], [1.1163287829994629e-5; 3.329966827474876e-6; … ; 2.7831576005301546e-5; -6.445545478287514e-5;;], [-2.2814900705447945e-5; -2.2186519798230124e-6; … ; 3.270269975138833e-5; 4.074071048659517e-5;;], [-4.3680174468771416e-5; -7.878362607310794e-8; … ; -1.4005399961224824e-5; 8.596692063289092e-5;;], [5.3492529869621306e-5; -4.0090216882695835e-7; … ; -1.8146955448447057e-5

In [141]:
@time solveStrain(u)

  1.526969 seconds (13.82 M allocations: 1015.001 MiB, 14.99% gc time)


elementwise TensorField
[[3.3831020691491985e-7; 5.410474984667013e-7; … ; -1.1957431261522744e-5; 3.5564953637996126e-5;;], [2.9870183876189594e-6; 1.8592654330836978e-5; … ; -1.9107435703011896e-5; 4.266973601557373e-5;;], [1.3064995123031418e-5; -2.8819977457603813e-5; … ; -2.5032479062644074e-5; -0.00019841780115441732;;], [1.712737851702032e-5; 4.5873824626422335e-6; … ; -1.1333005651417527e-6; 6.447111163459867e-7;;], [-6.82533436350216e-6; 2.332821502620125e-7; … ; -1.9528692799381004e-5; 0.0001456222097888215;;], [-6.186684693512123e-6; 5.102796813566401e-7; … ; -1.918832683869983e-5; 0.00015913039930430658;;], [1.1163287829994635e-5; 3.3299668274748623e-6; … ; 2.7831576005301532e-5; -6.445545478287514e-5;;], [-2.281490070544796e-5; -2.218651979823035e-6; … ; 3.270269975138837e-5; 4.0740710486595277e-5;;], [-4.36801744687714e-5; -7.878362609514689e-8; … ; -1.4005399961226125e-5; 8.596692063289006e-5;;], [5.349252986962128e-5; -4.0090216882641624e-7; … ; -1.8146955448446948e-5; 

In [142]:
showElementResults(A, name="strain")

2

In [143]:
I = TensorField(prob, "part", [1 0 0; 0 1 0; 0 0 1])
S = 2μ * A + λ * trace(A) * I;

In [144]:
showDoFResults(S, name="stress")

3

In [145]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.15.0-git
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib tinyobjloader
FLTK version  : 1.3.8
OCC version   : 7.9.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [146]:
gmsh.finalize()